02 - Entrega

Objetivo: En el notebook anterior (01 - exploración) se identificaron las columnas con valores faltantes y las variables categóricas principales. En este notebook se aplicarán las transformaciones necesarias para limpiar y preparar los datos.

# Cargamos el Train.csv

In [ ]:
!pip install -q kaggle
import os

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"juanjosemonsalveh","key":"365c6fb1cb1ec7b8744fd8e8c630ea75"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list | head

ref                                                            title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset   Global Earthquake-Tsunami Risk Assessment Dataset        16151  2025-10-01 16:35:53.273000          13265        459  1.0              
jockeroika/life-style-data                                     Life Style Data                                        3995645  2025-10-14 13:50:45.303000          15578        326  0.8235294        
jaderz/hospital-beds-management                                Hospital Beds Management                                 47583  2025-10-03 09:21:58.590000          10871        280  1.0              
ahmad

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -p ./data
!unzip -qo "./data/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip" -d "./data"

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 2.58GB/s]


In [ ]:
!ls ./data

submission_example.csv	train.csv
test.csv		udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip


In [ ]:
import pandas as pd
df = pd.read_csv("./data/train.csv")
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# Identificacion de tipo de datos y porcentaje de valores nulos de cada columna


In [ ]:
# Revisión de estructura
print("Filas, Columnas:", df.shape)
print("\nTipos de datos:\n", df.dtypes)

# Revisión de valores nulos
null_percent = df.isna().mean().sort_values(ascending=False) * 100
print("\nPorcentaje de valores nulos: \n", null_percent.head(21))

Filas, Columnas: (692500, 21)

Tipos de datos:
 ID                               int64
PERIODO_ACADEMICO                int64
E_PRGM_ACADEMICO                object
E_PRGM_DEPARTAMENTO             object
E_VALORMATRICULAUNIVERSIDAD     object
E_HORASSEMANATRABAJA            object
F_ESTRATOVIVIENDA               object
F_TIENEINTERNET                 object
F_EDUCACIONPADRE                object
F_TIENELAVADORA                 object
F_TIENEAUTOMOVIL                object
E_PRIVADO_LIBERTAD              object
E_PAGOMATRICULAPROPIO           object
F_TIENECOMPUTADOR               object
F_TIENEINTERNET.1               object
F_EDUCACIONMADRE                object
RENDIMIENTO_GLOBAL              object
INDICADOR_1                    float64
INDICADOR_2                    float64
INDICADOR_3                    float64
INDICADOR_4                    float64
dtype: object

Porcentaje de valores nulos: 
 F_TIENEAUTOMOVIL               6.299350
F_TIENELAVADORA                5.743394
F_TIENE

# Separamos la columna tarjet (RENDIMIENTO_GLOBAL) de los features

Esto se hace para poder trabajar sobre el modelo sin modificar nuestra columna objetivo, la cual se debe de mantener intacta

In [ ]:
target_col = "REN1DIMIENTO_GLOBAL"
X = df.drop(columns=[target_col]) #Features
y = df[target_col] #Tarjet

# Identificacion de tipo de dato de las columnas feature

Los modelos solo entienden numeros, pero nuestro Train contiene 3 tipos de datos:

**Numericos:**

- Debemos escalarlos (Transformar los valores de las variables numéricas en un conjunto de datos para que estén dentro de un rango similar o estándar)

- imputarlos (Reemplazar valores faltantes)

**Categoricos:**

- Codificarlos (Transformar datos que representan grupos o etiquetas en un formato numérico)

**Identificadores:**
- Eliminarlos (No son relevantes para nuestro estudio)

In [ ]:
import numpy as np

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numéricas:", len(numeric_features))
print("Categóricas:", len(categorical_features))

Numéricas: 6
Categóricas: 14


In [ ]:
print(df[numeric_features].isna().sum().sum(), "valores nulos numéricos")
print(df[categorical_features].isna().sum().sum(), "valores nulos categóricos")

0 valores nulos numéricos
297378 valores nulos categóricos


# Pipeline de procesamiento

sirve para automatizar todos los pasos de limpieza y transformación de datos antes de entrenar un modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import inspect


# 1. Limpieza inicial de columnas

# Eliminar duplicado (F_TIENEINTERNET.1)
# Esta columna repite la misma información que F_TIENEINTERNET
if "F_TIENEINTERNET.1" in X.columns:
    X = X.drop(columns=["F_TIENEINTERNET.1"])
    print("Columna duplicada 'F_TIENEINTERNET.1' eliminada")

# Eliminar identificadores (no aportan al modelo)
id_like = [c for c in X.columns if c.strip().lower() in ("id", "id_estudiante", "id_alumno")]
if id_like:
    X = X.drop(columns=id_like)
    print("Columnas de identificación eliminadas:", id_like)

# Mover numéricos de baja cardinalidad a categóricos
low_card_as_cat = [c for c in numeric_features if X_train[c].nunique() <= 10]
for c in low_card_as_cat:
    numeric_features.remove(c)
    categorical_features.append(c)

print("Columnas numéricas de baja cardinalidad reclasificadas:", low_card_as_cat)

# 2. Dividimos el total de los datos 80% para entrenar y 20% para probar
try:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
except ValueError:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
print("Tamaño entrenamiento:", X_train.shape)
print("Tamaño validación:", X_val.shape)

# 3. Identificamos tipos de columnas que se tiene despues de los cambios
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

print("Columnas numéricas:", len(numeric_features))
print("Columnas categóricas:", len(categorical_features))

# Análisis de valores nulos
print("Valores nulos numéricos:", df[numeric_features].isna().sum().sum())
print("Valores nulos categóricos:", df[categorical_features].isna().sum().sum())


# 4. OneHot
oh_kwargs = dict(handle_unknown="ignore", dtype=np.float32)
if "sparse_output" in inspect.signature(OneHotEncoder).parameters:
    oh_kwargs["sparse_output"] = False
else:
    oh_kwargs["sparse"] = False
if "min_frequency" in inspect.signature(OneHotEncoder).parameters:
    oh_kwargs["min_frequency"] = 40
    oh_kwargs["handle_unknown"] = "infrequent_if_exist"

# 5. Pipelines
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(**oh_kwargs))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

# 6. Aplicar las transformaciones
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed   = preprocessor.transform(X_val)

# Obtener los nombres de las columnas codificadas
oh = preprocessor.named_transformers_['cat'].named_steps['encoder']
cat_names = oh.get_feature_names_out(categorical_features)
all_names = np.concatenate([numeric_features, cat_names])

print("Shape X_train procesado:", X_train_processed.shape)
print("Shape X_val procesado:", X_val_processed.shape)

Tamaño entrenamiento: (554000, 18)
Tamaño validación: (138500, 18)
Columnas numéricas: 4
Columnas categóricas: 14
Valores nulos numéricos: 0
Valores nulos categóricos: 270749
Columnas numéricas de baja cardinalidad reclasificadas: []
Shape X_train procesado: (554000, 730)
Shape X_val procesado: (138500, 730)


In [ ]:
print(np.isnan(X_train_processed).sum(), "NaN en X_train procesado")

0 NaN en X_train procesado


**Anlisis de resultado**
Shape X_train
554 000 filas: son los registros que el modelo va a usar para aprender patrones.

730 columnas: son las variables (ya transformadas) que describen cada estudiante.

Shape X_val:
138 500 filas: son ejemplos que el modelo nunca ve durante el entrenamiento.

730 columnas: mismas variables, procesadas igual que las del train.



# Guardamos y descargamos los resultados

In [ ]:
pd.DataFrame(X_train_processed, columns=all_names).to_csv("train_preprocessed.csv", index=False)
pd.DataFrame(X_val_processed,   columns=all_names).to_csv("val_preprocessed.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_val.to_csv("y_val.csv", index=False)

In [ ]:
from google.colab import files
files.download("train_preprocessed.csv")